anova using sklearn

In [1]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [16]:
iris = load_iris()
X = iris.data
y = iris.target

In [22]:
anova = SelectKBest(f_classif, k=2)
X_selected = anova.fit_transform(X, y)

In [23]:
anova.scores_, anova.pvalues_

(array([ 119.26450218,   49.16004009, 1180.16118225,  960.0071468 ]),
 array([1.66966919e-31, 4.49201713e-17, 2.85677661e-91, 4.16944584e-85]))

In [24]:
import numpy as np

In [32]:
classes = np.unique(y)
n_classes = len(classes)
n_samples ,n_features = X.shape

In [33]:
groups = {key: [] for key in classes}
 
for i in range(n_samples):
    groups[y[i]].append(X[i,:])
    
groups = {key: np.array(val) for key,val in groups.items()}

In [34]:
def compute_means(groups):
    means = {key: [] for key in classes}
    for y_, x_ in groups.items():
        cols = [groups[y_][:,i] for i in range(n_features)]
        means[y_] = [col.mean() for col in cols]
    grand_mean = []
    for i in range(n_features):
        grand_mean.append(X[:,i].mean())
    return means, grand_mean

In [35]:
def get_btw(groups, means, grand_mean):
    SS_btw = np.zeros(n_features)
    for y_, x_ in groups.items():
        for i in range(n_features):
            SS_btw[i] += len(x_)*(means[y_][i]-grand_mean[i])**2

    n_classes = len(groups)
    df_btw = n_classes - 1
    MS_btw = SS_btw/df_btw
    return SS_btw, MS_btw

In [36]:
def get_wthn(groups, means, grand_mean):
    SS_wthn = np.zeros(n_features)
    for y_, x_ in groups.items():
        for i in range(n_features):
            SS_wthn[i] += sum((x_[:,i] - means[y_][i])**2)
    n_classes = len(groups)
    df_wthn = n_samples - n_classes
    MS_wthn = [SS/df_wthn for SS in SS_wthn]
    return SS_wthn, MS_wthn

In [37]:
def get_fvalues(groups):
    means, grand_mean = compute_means(groups)
    _, MS_btw = get_btw(groups, means, grand_mean)
    _, MS_wthn = get_wthn(groups, means, grand_mean)
    F = MS_btw/MS_wthn
    return F

In [41]:
def select_k_features(groups,k):
    F = get_fvalues(groups)
    indices = (-F).argsort()[:k]
    return indices

In [42]:
get_fvalues(groups)

array([ 119.26450218,   49.16004009, 1180.16118225,  960.0071468 ])

In [43]:
select_k_features(groups,2)

array([2, 3], dtype=int64)